In [ ]:
import numpy as np

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

In [ ]:
!wget https://raw.githubusercontent.com/raeubaen/RadioMonteCarlo/main/tf_keras_model.py

In [ ]:
!wget dati....

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net, get_particle_net_lite, edge_conv

In [ ]:
data_folder = "../hits_data/keras_input"

name_map = {"points": "rphi", "features": "et", "mask": "mask"} # va provato sia xy che rphi


nlim = 200000 # sta cazzo di ram da poveri porco tutto ma come cazzo dobbiamo fare


signal = {key: np.load(f"{data_folder}/signal_{name_map[key]}.npy")[:nlim] for key in name_map}

bkg    = {key: np.load(f"{data_folder}/bkg_{   name_map[key]}.npy")[:nlim] for key in name_map}

In [ ]:
S = signal["mask"].shape[0]
B =    bkg["mask"].shape[0]

N = S + B

p = np.random.permutation(S+B)

print(p)

[237000 209529 119499 ...  66218 260195 290670]


In [ ]:
data = {key: np.concatenate( (signal[key], bkg[key]) )[p] for key in name_map}

label = np.concatenate( ( np.ones( (S) ), np.zeros( (B) ) ) )[p]

In [ ]:
np.sum(label)/len(label)

0.4307638011316416

In [ ]:
train_data  = {key: data[key][int(N*0.0) : int(N*0.7)] for key in name_map}
test_data   = {key: data[key][int(N*0.7) : int(N*0.9)] for key in name_map}
val_data    = {key: data[key][int(N*0.9) : int(N*1.0)] for key in name_map}

train_label = label[int(N*0.0) : int(N*0.7)]
test_label  = label[int(N*0.7) : int(N*0.9)]
val_label   = label[int(N*0.9) : int(N*1.0)]

In [ ]:
model_type = 'particle_net_lite' # choose between 'particle_net' and 'particle_net_lite'
num_classes = 2
input_shapes = {'points': (50, 2), 'features': (50, 2), 'mask':(50, 1)}

if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes, input_shapes)

2022-06-12 18:30:52.867168: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ruben/root/lib:/home/ruben/geant4/geant4.10.07.p02-install/lib:/usr/lib/x86_64-linux-gnu
2022-06-12 18:30:52.867197: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-12 18:30:52.867215: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ruben): /proc/driver/nvidia/version does not exist
2022-06-12 18:30:52.867460: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-12 18:30:52.901118: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2099940000 Hz
2022-06-12 18:30:52.901954: I tensorflow/compiler/xla/se

In [ ]:
# Training parameters
batch_size = 1024 if 'lite' in model_type else 4096 # 384
epochs = 30

In [ ]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

[2022-06-12 18:30:53,543] INFO: Learning rate: 0.001000


Model: "ParticleNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mask (InputLayer)               [(None, 50, 1)]      0                                            
__________________________________________________________________________________________________
tf_op_layer_NotEqual (TensorFlo [(None, 50, 1)]      0           mask[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_Cast (TensorFlowOpL [(None, 50, 1)]      0           tf_op_layer_NotEqual[0][0]       
__________________________________________________________________________________________________
tf_op_layer_Equal (TensorFlowOp [(None, 50, 1)]      0           tf_op_layer_Cast[0][0]           
________________________________________________________________________________________

In [ ]:
# Prepare model model saving directory.
import os
save_dir = 'model_checkpoints'
model_name = '%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler, progress_bar]

In [ ]:
model.fit(
    [train_data["points"], train_data["features"], train_data["mask"]], 
    train_label,
    batch_size=batch_size,
    epochs=1, # --- train only for 1 epoch here for demonstration ---
    validation_data=([val_data["points"], val_data["features"], val_data["mask"]], val_label),
    shuffle=True,
    #callbacks=callbacks
)

2022-06-12 18:30:53.658895: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 98377200 exceeds 10% of free system memory.
2022-06-12 18:30:53.758906: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 98377200 exceeds 10% of free system memory.
2022-06-12 18:30:55.533447: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 45875200 exceeds 10% of free system memory.
2022-06-12 18:30:55.539819: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 45875200 exceeds 10% of free system memory.
2022-06-12 18:30:55.587279: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 45875200 exceeds 10% of free system memory.


241/241 [==============================] - 458s 2s/step - loss: 0.4905 - accuracy: 0.7606 - val_loss: 0.5050 - val_accuracy: 0.7710
